### LIBRERÍAS

In [2]:
# Librerías básicas
import numpy as np
import pandas as pd

# Librerías de Preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn import metrics 


#Librerís de Modelado y evaluación
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### DATOS SÁTELITALES

In [3]:

satelitales = pd.read_csv('../Data/Boyaca_NDVI_EVI_SoilMoisture_MENSUAL_2005_2025_CLEAN.csv')
satelitales



,EVI,NDVI,fecha,year,month,yyyymm
0,0.381304,0.688825,2005-01-01,2005,1,2005-01
1,0.362783,0.668144,2005-02-01,2005,2,2005-02
2,0.379677,0.632942,2005-03-01,2005,3,2005-03
3,0.435677,0.667824,2005-04-01,2005,4,2005-04
4,0.405432,0.695091,2005-05-01,2005,5,2005-05
...,...,...,...,...,...,...
243,0.419982,0.695208,2025-04-01,2025,4,2025-04
244,0.434576,0.731797,2025-05-01,2025,5,2025-05
245,0.405199,0.743507,2025-06-01,2025,6,2025-06
246,0.453621,0.732904,2025-07-01,2025,7,2025-07


In [4]:
satelitales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   EVI     248 non-null    float64
 1   NDVI    248 non-null    float64
 2   fecha   248 non-null    object 
 3   year    248 non-null    int64  
 4   month   248 non-null    int64  
 5   yyyymm  248 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 11.8+ KB


In [5]:
satelitales.describe()

,EVI,NDVI,year,month
count,248.000000,248.000000,248.000000,248.000000
mean,0.415884,0.690760,2014.838710,6.435484
std,0.030513,0.031170,5.980247,3.445920
min,0.322249,0.547600,2005.000000,1.000000
25%,0.396819,0.674575,2010.000000,3.000000
50%,0.419184,0.697605,2015.000000,6.000000
75%,0.437864,0.712912,2020.000000,9.000000
max,0.499423,0.746885,2025.000000,12.000000


### ACOTACIÓN TIEMPO Y CORRECIÓN FECHA

In [6]:


# 1) Copia y normaliza nombres
df_ndvi = satelitales.copy()
df_ndvi.columns = df_ndvi.columns.str.strip().str.lower()

# 2) Detecta columna de fecha y pásala a datetime
fecha_col = 'fecha' if 'fecha' in df_ndvi.columns else [c for c in df_ndvi.columns if 'time' in c or 'fecha' in c][0]
df_ndvi[fecha_col] = pd.to_datetime(df_ndvi[fecha_col], errors='coerce')

# 3) Filtra 2012–2019
df_ndvi = df_ndvi[(df_ndvi[fecha_col].dt.year >= 2012) & (df_ndvi[fecha_col].dt.year <= 2019)]

# 4) Quédate solo con fecha + NDVI/EVI (en cualquier variante de nombre)
idx_cols = [c for c in df_ndvi.columns if c == 'ndvi' or c == 'evi' or c.startswith('ndvi') or c.startswith('evi')]
df_ndvi = df_ndvi[[fecha_col] + idx_cols].sort_values(fecha_col).reset_index(drop=True)
# (Opcional) Si hubiera filas repetidas por fecha:
# df_ndvi = df_ndvi.drop_duplicates(subset=[fecha_col])

# 5) Campos auxiliares
df_ndvi['year'] = df_ndvi[fecha_col].dt.year
df_ndvi['month'] = df_ndvi[fecha_col].dt.month
df_ndvi['yyyymm'] = df_ndvi[fecha_col].dt.strftime('%Y-%m')

print("NDVI/EVI listo:", df_ndvi.shape)
display(df_ndvi.head())


NDVI/EVI listo: (96, 6)


,fecha,evi,ndvi,year,month,yyyymm
0,2012-01-01,0.398118,0.699554,2012,1,2012-01
1,2012-02-01,0.390345,0.671628,2012,2,2012-02
2,2012-03-01,0.376592,0.643635,2012,3,2012-03
3,2012-04-01,0.451125,0.697795,2012,4,2012-04
4,2012-05-01,0.447870,0.697231,2012,5,2012-05


### RENDIMIENTOS CULTIVO CAFÉ

In [7]:
rendimientos = pd.read_excel('../Data/Serie rendimiento cafe 2012 - 2019.xlsx')
rendimientos


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Regresar al índice
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Encuesta Nacional Agropecuaria ENA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Área sembrada, cosechada, producción y rendimi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,Nota: (-) No existe dato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,Nota: por aproximación decimal se pueden prese...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,"Nota: para el periodo 2012 - 2015, el Total Na...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,Nota: Para la estimación de áreas de café en e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
rendimientos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          44 non-null     object 
 1   Unnamed: 1          115 non-null    object 
 2   Unnamed: 2          116 non-null    object 
 3   Unnamed: 3          115 non-null    float64
 4   Unnamed: 4          115 non-null    float64
 5   Unnamed: 5          115 non-null    float64
 6   Unnamed: 6          115 non-null    float64
 7   Unnamed: 7          115 non-null    float64
 8   Unnamed: 8          115 non-null    float64
 9   Unnamed: 9          115 non-null    float64
 10  Unnamed: 10         0 non-null      float64
 11  Unnamed: 11         116 non-null    object 
 12  Unnamed: 12         115 non-null    float64
 13  Unnamed: 13         115 non-null    float64
 14  Unnamed: 14         115 non-null    float64
 15  Unnamed: 15         115 non-null    float64
 16  Unnamed:

In [9]:
rendimientos.describe()

,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 12,Unnamed: 13,...,Unnamed: 27,Unnamed: 28,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Regresar al índice
count,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,0.0,115.000000,115.000000,...,115.000000,0.0,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,115.000000,0.0
mean,21117.294482,21685.907447,21952.915626,20519.291125,24483.297105,24538.417950,25246.191631,NaN,16286.663108,17025.360206,...,25201.955950,NaN,20.401138,19.456283,19.644288,19.915392,19.803126,23.707466,20.258662,NaN
std,88237.037991,91212.532788,92746.969088,85057.889116,96778.827458,96625.799026,99391.776867,NaN,67427.150605,70821.585484,...,97605.099195,NaN,187.609198,187.667196,187.776032,187.845676,187.976392,188.520530,188.076750,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,...,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,0.000000,0.000000,0.000000,0.000000,11.607164,11.200912,NaN,0.000000,0.000000,...,12.106434,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.085000,NaN
50%,14.120917,12.855257,12.629662,26.345579,61.000000,99.647541,94.280904,NaN,15.642095,12.790667,...,97.896670,NaN,0.094436,0.143064,0.076159,0.135690,0.159550,0.752524,0.728187,NaN
75%,7196.616084,7507.001061,6167.122563,6505.613358,10188.030130,11031.475000,11551.055000,NaN,6228.708823,7006.402269,...,13514.440000,NaN,1.263521,1.203327,1.224977,1.284163,1.665000,2.595000,1.936488,NaN
max,728530.891460,757744.246488,768140.300583,711010.951995,814808.260000,815103.180000,839660.630000,NaN,559477.973916,592251.436292,...,820614.180000,NaN,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000,NaN


### DATOS ANUALES - CONVERSIÓN A MENSUALES

In [10]:
# Datos anuales de rendimiento de café en Boyacá (t/ha) - DIVISIÓN MENSUAL

# --- Datos anuales de Boyacá (t/ha) ---
rend_anual = {
    2012: 0.5,
    2013: 0.9,
    2014: 0.8,
    2015: 0.6,
    2016: 1.0,
    2017: 1.1,
    2018: 1.0,
    2019: 0.9
}

# --- Distribución estacional mensual ---
# Normalizada para que sume 1 (porcentajes de rendimiento)
pattern = np.array([
    0.04, 0.05, 0.06, 0.08, 0.10, 0.12, 0.08, 0.06, 0.07, 0.10, 0.14, 0.10
])
pattern = pattern / pattern.sum()

# --- Generar dataset mensual ---
rows = []
for year, total in rend_anual.items():
    for month in range(1, 13):
        fecha = pd.Timestamp(year=year, month=month, day=1)
        rows.append({
            "departamento": "Boyacá",
            "fecha": fecha,
            "anio": year,
            "month": month,
            "yyyymm": fecha.strftime("%Y-%m"),
            "rendimiento_t_ha": round(total * pattern[month - 1], 3)
        })

df_rend_boyaca = pd.DataFrame(rows)

# --- Verificar proporciones ---
df_check = df_rend_boyaca.groupby("anio")["rendimiento_t_ha"].sum().round(3)
print("Comprobación de sumas anuales:")
print(df_check)

# --- Mostrar primeros meses ---
df_rend_boyaca.head(24)



Comprobación de sumas anuales:
anio
2012    0.5
2013    0.9
2014    0.8
2015    0.6
2016    1.0
2017    1.1
2018    1.0
2019    0.9
Name: rendimiento_t_ha, dtype: float64


,departamento,fecha,anio,month,yyyymm,rendimiento_t_ha
0,Boyacá,2012-01-01,2012,1,2012-01,0.020
1,Boyacá,2012-02-01,2012,2,2012-02,0.025
2,Boyacá,2012-03-01,2012,3,2012-03,0.030
3,Boyacá,2012-04-01,2012,4,2012-04,0.040
4,Boyacá,2012-05-01,2012,5,2012-05,0.050
5,Boyacá,2012-06-01,2012,6,2012-06,0.060
6,Boyacá,2012-07-01,2012,7,2012-07,0.040
7,Boyacá,2012-08-01,2012,8,2012-08,0.030
8,Boyacá,2012-09-01,2012,9,2012-09,0.035
9,Boyacá,2012-10-01,2012,10,2012-10,0.050



Cada año se descompone en 12 meses, respetando la estacionalidad típica del café:

Mitaca: abril–junio con picos leves.

Principal: octubre–diciembre con picos fuertes.

Enero–marzo y julio–septiembre con actividad moderada/baja.

### SIMULACIÓN VARIABLES CLIMÁTICAS

In [11]:

#  SIMULACIÓN DE VARIABLES CLIMÁTICAS (Precipitación, Temperatura, Humedad)


# Crear rango de fechas 2012-2019 mensual
fechas = pd.date_range(start="2012-01-01", end="2019-12-31", freq="MS")

np.random.seed(42)  # reproducibilidad

# Simulación climática realista
precipitacion = np.random.normal(loc=160, scale=40, size=len(fechas))  # mm/mes
temp_max = np.random.normal(loc=25, scale=1.5, size=len(fechas))       # °C
temp_min = np.random.normal(loc=15, scale=1.2, size=len(fechas))       # °C
humedad = 100 - (temp_max - 20)*4 + np.random.normal(0, 3, len(fechas))  # relación inversa

df_clima = pd.DataFrame({
    "fecha": fechas,
    "Precipitacion": np.clip(precipitacion, 60, 300),
    "TempMax": np.clip(temp_max, 20, 30),
    "TempMin": np.clip(temp_min, 10, 20),
    "HumedadRelativa": np.clip(humedad, 60, 90)
})

print("Variables climáticas simuladas:", df_clima.shape)
display(df_clima.head())

Variables climáticas simuladas: (96, 5)


,fecha,Precipitacion,TempMax,TempMin,HumedadRelativa
0,2012-01-01,179.868566,25.444180,15.256912,79.066254
1,2012-02-01,154.469428,25.391583,13.505113,76.565570
2,2012-03-01,185.907542,25.007670,15.207817,79.344953
3,2012-04-01,220.921194,24.648119,15.462381,79.928520
4,2012-05-01,150.633865,22.876944,13.939371,86.724130


### INTEGRACIÓN DATAFRAMES

In [12]:
#INTEGRACIÓN 

# yyyymm de cada fuente
df_ndvi["yyyymm"]       = df_ndvi[fecha_col].dt.strftime("%Y-%m")
df_rend_boyaca["yyyymm"]= df_rend_boyaca["fecha"].dt.strftime("%Y-%m")
df_clima["yyyymm"]      = df_clima["fecha"].dt.strftime("%Y-%m")

# Merge por yyyymm
df_modelo = (
    df_ndvi.merge(df_rend_boyaca[["yyyymm","rendimiento_t_ha"]], on="yyyymm", how="inner")
           .merge(df_clima.drop(columns=["fecha"], errors="ignore"), on="yyyymm", how="left")
           .rename(columns={"rendimiento_t_ha":"rendimiento"})
           .assign(fecha=lambda d: pd.to_datetime(d["yyyymm"]))
           .assign(anio=lambda d: d["fecha"].dt.year, month=lambda d: d["fecha"].dt.month)
           .sort_values("fecha", kind="stable")
           .reset_index(drop=True)
)

# (opcional) Reordenar para visualización rápida
cols = ["fecha","yyyymm","ndvi", "evi", "rendimiento","Precipitacion","TempMax","TempMin","HumedadRelativa"]
df_modelo = df_modelo[[c for c in cols if c in df_modelo.columns]]

# Mostrar tabla limpia
print("Dataset mensual para modelado:", df_modelo.shape)
display(df_modelo.head(12))

Dataset mensual para modelado: (96, 9)


,fecha,yyyymm,ndvi,evi,rendimiento,Precipitacion,TempMax,TempMin,HumedadRelativa
0,2012-01-01,2012-01,0.699554,0.398118,0.020,179.868566,25.444180,15.256912,79.066254
1,2012-02-01,2012-02,0.671628,0.390345,0.025,154.469428,25.391583,13.505113,76.565570
2,2012-03-01,2012-03,0.643635,0.376592,0.030,185.907542,25.007670,15.207817,79.344953
3,2012-04-01,2012-04,0.697795,0.451125,0.040,220.921194,24.648119,15.462381,79.928520
4,2012-05-01,2012-05,0.697231,0.447870,0.050,150.633865,22.876944,13.939371,86.724130
5,2012-06-01,2012-06,0.710103,0.415739,0.060,150.634522,24.369032,15.184470,85.072678
6,2012-07-01,2012-07,0.704582,0.441602,0.040,223.168513,24.485928,15.069850,83.127334
7,2012-08-01,2012-08,0.700551,0.423252,0.030,190.697389,23.796584,13.628436,82.734935
8,2012-09-01,2012-09,0.691045,0.397087,0.035,141.221025,24.758071,15.429345,83.666514
9,2012-10-01,2012-10,0.702355,0.421132,0.050,181.702402,25.606076,15.672941,78.497593


### PREPROCESAMIENTO

In [13]:
# PREPROCESAMIENTO SIMPLE

# 1) Copia y asegura columnas de tiempo
datos = df_modelo.copy()

# fecha ↔ yyyymm
if "fecha" not in datos.columns:
    if "yyyymm" in datos.columns:
        datos["fecha"] = pd.to_datetime(datos["yyyymm"], format="%Y-%m", errors="coerce")
    else:
        raise ValueError("Falta 'fecha' o 'yyyymm' en df_modelo.")
datos["mes"]  = datos["fecha"].dt.month
datos["anio"] = datos["fecha"].dt.year

# 2) Estandariza nombres NDVI/EVI y clima (tolerante a mayúsculas/minúsculas)
renombres = {}
for c in datos.columns:
    cl = c.strip().lower()
    if cl.startswith("ndvi"): renombres[c] = "NDVI"
    if cl.startswith("evi"):  renombres[c] = "EVI"
    if cl in {"precipitacion"}: renombres[c] = "Precipitacion"
    if cl in {"tmax","tempmax"}: renombres[c] = "TempMax"
    if cl in {"tmin","tempmin"}: renombres[c] = "TempMin"
    if cl in {"humedad","humedadrelativa"}: renombres[c] = "HumedadRelativa"
datos = datos.rename(columns=renombres)

# 3) Estacionalidad mensual (seno/coseno)
datos["mes_sin"] = np.sin(2*np.pi*datos["mes"]/12)
datos["mes_cos"] = np.cos(2*np.pi*datos["mes"]/12)

# 4) Selección de variables (toma solo las que existan)
variables_base = ["NDVI","EVI","Precipitacion","TempMax","TempMin","HumedadRelativa"]
variables_modelo = [v for v in variables_base if v in datos.columns] + ["mes_sin","mes_cos"]

X = datos[variables_modelo].apply(pd.to_numeric, errors="coerce")   # predictores
y = pd.to_numeric(datos["rendimiento"], errors="coerce")            # objetivo

# 5) División temporal y transformación (imputar + escalar)
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(
    X, y, test_size=0.20, shuffle=False
)

prepro = Pipeline([
    ("imputar", SimpleImputer(strategy="median")),
    ("escalar", StandardScaler())
])

X_entrena_prep = prepro.fit_transform(X_entrena)
X_prueba_prep  = prepro.transform(X_prueba)

print("Variables usadas:", variables_modelo)
print("Variable objetivo:", "rendimiento")
print("Train:", X_entrena_prep.shape, " | Test:", X_prueba_prep.shape)


Variables usadas: ['NDVI', 'EVI', 'Precipitacion', 'TempMax', 'TempMin', 'HumedadRelativa', 'mes_sin', 'mes_cos']
Variable objetivo: rendimiento
Train: (76, 8)  | Test: (20, 8)


## 🌎 ¿Por qué usamos `mes_sin` y `mes_cos`?

En datos **mensuales** como los de la caficultura, el **mes del año influye fuertemente** en el rendimiento (por lluvias, floración, cosecha, etc.).  
Sin embargo, el número del mes (`1–12`) es una **variable cíclica**, no lineal.

### 🧩 Problema
Si usas el número del mes directamente, el modelo interpretará que:

> Diciembre (12) está muy lejos de Enero (1),

cuando en realidad **son adyacentes** en el ciclo anual.

### 🧭 Solución: *codificación cíclica*

Transformamos el número del mes en **coordenadas circulares** usando funciones seno y coseno.  
Así, los meses se representan sobre un círculo, donde diciembre y enero “se tocan”.

### 📐 Fórmulas

\[
\text{mes\_sin} = \sin\left(2\pi \times \frac{\text{mes}}{12}\right)
\]

\[
\text{mes\_cos} = \cos\left(2\pi \times \frac{\text{mes}}{12}\right)
\]

### 🌀 Ejemplos

| Mes | mes_sin | mes_cos | Interpretación |
|-----|----------|----------|----------------|
| Enero | ≈ 0.5 | ≈ 0.87 | Inicio de año |
| Julio | ≈ -1 | ≈ 0 | Mitad del año |
| Diciembre | ≈ 0 | ≈ 1 | Fin del año |

### 💡 Beneficio

El modelo aprende **la estacionalidad continua**, es decir, los cambios **graduales entre meses**,  
sin los saltos artificiales que ocurren si se usan números enteros.


### MODELADO Y SIMULACIÓN

In [19]:
#  MODELADO Y EVALUACIÓN 

# Modelos a probar
modelos = {
    "Regresión Lineal": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42)
}

resultados = []

# Entrenar y evaluar cada modelo
for nombre, modelo in modelos.items():
    modelo.fit(X_entrena_prep, y_entrena)
    y_pred = modelo.predict(X_prueba_prep)
    
    mae = mean_absolute_error(y_prueba, y_pred)
    rmse = np.sqrt(mean_squared_error(y_prueba, y_pred))
    r2 = r2_score(y_prueba, y_pred)
    
    resultados.append([nombre, mae, rmse, r2])

# Tabla de resultados
df_resultados = pd.DataFrame(resultados, columns=["Modelo", "MAE", "RMSE", "R2"]).sort_values("R2", ascending=False)

print("📊 Resultados del modelado:")
display(df_resultados)

# Mejor modelo
mejor_modelo = df_resultados.iloc[0, 0]
print(f"✅ Mejor modelo: {mejor_modelo}")


📊 Resultados del modelado:


,Modelo,MAE,RMSE,R2
1,Random Forest,0.012968,0.018749,0.524479
0,Regresión Lineal,0.016795,0.022187,0.334078
2,Gradient Boosting,0.016886,0.022566,0.311165


✅ Mejor modelo: Random Forest


### ESTRUCTURAR PIPELINE PARA CREACIÓN DE TABLERO

In [20]:
# ================= PRODUCCIÓN: reentrenar el mejor pipeline =================
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# 1) Partimos del dataframe ya preprocesado "datos" (el del bloque anterior)
#    Si no lo tienes, puedes usar df_modelo y repetir el preprocesamiento simple que hicimos.

# a) Seleccionar SOLO columnas numéricas (descarta fechas/strings automáticamente)
cols_numericas = datos.select_dtypes(include=[np.number]).columns

# b) Separar X, y (y = rendimiento)
cols_numericas = cols_numericas.drop("rendimiento")   # quitar el target
X_full = datos[cols_numericas].copy()
y_full = datos["rendimiento"].astype(float)

# 2) Pipeline final (imputar + escalar + modelo ganador)
prepro_final = Pipeline([
    ("imputar", SimpleImputer(strategy="median")),
    ("escalar", StandardScaler())
])

# Si ya elegiste el mejor modelo en df_resultados:
modelos_dict = {
    "Regresión Lineal": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Linear": LinearRegression(),
    "RF": RandomForestRegressor(random_state=42),
    "GB": GradientBoostingRegressor(random_state=42),
}
best_name = df_resultados.sort_values("R2", ascending=False).iloc[0,0]
best_model = modelos_dict[best_name]

pipe = Pipeline([
    ("prepro", prepro_final),
    ("model", best_model)
])

# 3) Entrenar en TODO el histórico (listo para servir/predict)
pipe.fit(X_full, y_full)
print("✅ Pipeline final entrenado sin columnas datetime.")


✅ Pipeline final entrenado sin columnas datetime.


### CARGA DE ARTEFACTOS

In [25]:
import os, json, joblib
import numpy as np
import pandas as pd

os.makedirs("artifacts", exist_ok=True)

# (a) pipeline final ya entrenado en 'pipe'
joblib.dump(pipe, "artifacts/modelo_boyaca.pkl")

# (b) columnas/variables usadas por el pipeline
feature_cols = list(X_full.columns)  # si usaste X_full del flujo final
joblib.dump(feature_cols, "artifacts/feature_cols.pkl")

# (c) snapshot de métricas del notebook (busca la columna R2 sin importar cómo esté escrita)
col_r2   = [c for c in df_resultados.columns if 'r2'   in c.lower()][0]
col_rmse = [c for c in df_resultados.columns if 'rmse' in c.lower()][0]
col_mae  = [c for c in df_resultados.columns if 'mae'  in c.lower()][0]

meta = {
    "model": type(pipe.named_steps["model"]).__name__,
    "n_obs": int(len(y_full)),
    "features": feature_cols,
    "metricas_test_snapshot": {
        "R2":   float(df_resultados.sort_values(col_r2, ascending=False)[col_r2].iloc[0]),
        "RMSE": float(df_resultados.sort_values(col_r2, ascending=False)[col_rmse].iloc[0]),
        "MAE":  float(df_resultados.sort_values(col_r2, ascending=False)[col_mae].iloc[0]),
    }
}
with open("artifacts/metadata.json", "w") as f:
    json.dump(meta, f, indent=2)


In [26]:
# dataset mensual limpio para dashboards / BI
df_modelo.to_csv("artifacts/dataset_modelo.csv", index=False)

# resultados de modelos (ordenados por R2) para reportes
df_resultados.sort_values(col_r2, ascending=False).to_csv(
    "artifacts/df_resultados.csv", index=False
)


In [ ]:
pipe_loaded   = joblib.load("artifacts/modelo_boyaca.pkl")
feat_loaded   = joblib.load("artifacts/feature_cols.pkl")
with open("artifacts/metadata.json") as f:
    meta_loaded = json.load(f)
